In [109]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time 

# Change thesse file locations :) 

In [2]:
song_meta_table = pd.read_csv("merged_songs.csv")

In [3]:
# train_idx = test_idx = pd.read_csv("/Users/mbutler/Downloads/down_df.csv")
test_idx = pd.read_csv("/Users/mbutler/Downloads/test_idx.csv")

In [4]:
actual_data = pd.read_csv("full_playlist_wo_tracks.csv")

In [111]:
test = actual_data.loc[actual_data['pid'].isin(list(test_idx['pid']))]

In [100]:
client_id = 'b939139ade0a4698825027b82def40ca'
client_secret = 'f2389d5c42ef4baea58521293ff7ba74'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [101]:
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

      # meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']

      # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']

    track = [name, length, popularity, danceability, acousticness, danceability, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
    return track

In [137]:
def find_song_in_table(song_id,pos):
    try:
        return (list(song_meta_table.loc[song_meta_table['Unnamed: 0'] == song_id][['Song Name','popularity', 'duration_ms', 'danceability', 'energy',
           'key', 'loudness', 'mode', 'speechiness', 'acousticness',
           'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']].values[0]),pos)
    except:
        return song_id,pos
#     (our_songs.loc[our_songs['Unnamed: 0'] == song_id]['TrackID'],pos)



In [132]:
def put_song_in_playlist(playlist):        
    return [find_song_in_table(i[0],i[1]) for i in eval(playlist)]
    

In [134]:
def get_avgs(playlist):
    #Creates Avg columns for each playlist row
    count = 0
    none_count = 0
#     Features
    popularity = 0
    danceability = 0
    duration = 0
    energy = 0
    key = 0
    loudness = 0
    mode = 0
    speechiness = 0
    acousticness = 0
    instrumentalness = 0
    liveness = 0
    valence = 0
    tempo = 0
    time_signature = 0
    for i in playlist:
        if type(i[0]) != int:
            popularity += i[0][1] 
            duration += i[0][2] 
            danceability += i[0][3]
            energy += i[0][4]
            key += i[0][5]
            loudness += i[0][6]
            mode += i[0][7]
            speechiness += i[0][8]
            acousticness += i[0][9]
            instrumentalness += i[0][10]
            liveness += i[0][11]
            valence += i[0][12]
            tempo += i[0][13]
            time_signature += i[0][14]
        else:
            none_count += 1
        count += 1
        
#     if none_count == count:
        
#         num = np.random.randint(len(playlist))
#         sample = np.random.choice(len(playlist))
        
    
    
    return popularity/count,duration/count, danceability/count, energy/count, key/count,loudness/count, \
    mode/count,speechiness/count,acousticness/count,instrumentalness/count,liveness/count, \
    valence/count,tempo/count,time_signature/count
    

In [ ]:
%%time
test['tracks_trans'] = test['tracks'].apply(put_song_in_playlist)

In [107]:
test["avg_popularity"],test['avg_duration'],test["avg_danceability"],test['avg_energy'],test['avg_key'],test['avg_loudness'], \
test['avg_mode'],test['avg_speechiness'],test['avg_acousticness'] ,test['avg_instrumentalness'], \
test['avg_liveness'], test['avg_valence'],test['avg_temp'],test['avg_time_signature'] = zip(*test['tracks_trans'].map(get_avgs))


In [ ]:
test.to_csv("testing_data_avgs.csv")